# Previsão de atribuição de preços de imóveis

## Init 

In [3]:
import turicreate as tc
sales = tc.SFrame('home_data.sframe')
# Definir os nomes das colunas
str_columns = ['bathrooms', 'bedrooms', 'floors', 'zipcode', 'condition', 'view', 'grade', 'yr_built', 'yr_renovated']
int_columns = ['sqft_living', 'sqft_lot', 'price', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
bol_columns = ['waterfront']
# Aplicar conversões ao SFrame
for col in str_columns:
    sales[col] = sales[col].apply(str)

for col in int_columns:
    sales[col] = sales[col].apply(int)

for col in bol_columns:
    sales[col] = sales[col].apply(bool)
    

### 1. Seleção e estatísticas resumidas: Encontramos o código postal com o preço médio mais alto da casa. Qual é o preço médio das casas desse código postal?

In [5]:
zipcode_price_avg = sales.groupby('zipcode', {
    'average_price': tc.aggregate.MEAN('price')
})
zipcode_price_avg = zipcode_price_avg.sort('average_price', ascending=False)
top_zipcode = zipcode_price_avg[0]
print("CEP com o preço médio mais alto:", top_zipcode['zipcode'])
print("Preço médio:", top_zipcode['average_price'])

CEP com o preço médio mais alto: 98039
Preço médio: 2160606.6


### 2. Filtragem de dados: Que fração das casas tem área útil entre 2.000 pés quadrados e 4.000 pés quadrados?



In [21]:
# Filtrar as casas com área útil entre 2.000 e 4.000 pés quadrados
in_range = sales[(sales['sqft_living'] >= 2000) & (sales['sqft_living'] <= 4000)]
# Calcular o número total de casas
total_houses = sales.num_rows()
# Calcular o número de casas que atendem ao critério
in_range_count = len(in_range)
# Calcular a fração
fraction = in_range_count / total_houses
# Imprimir a fração
print("Fração de casas com área útil entre 2.000 e 4.000 pés quadrados:", fraction)

Fração de casas com área útil entre 2.000 e 4.000 pés quadrados: 0.4266413732475825


### 3. Criar um modelo de regressão com vários outros recursos: Qual é a diferença em RMSE entre o modelo treinado com my_features e o modelo treinado com advanced_features?

In [13]:
#Load dataset
train_data, test_data = sales.random_split(.8,seed=0)

In [16]:
# Modelo my_features
my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
my_features_model = tc.linear_regression.create(train_data,target='price',features=my_features,validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.016991     | 3763208.270523     | 181908.848367                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [15]:
# Modelo advanced_features
advanced_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                        'zipcode','condition', 'grade','waterfront','view','sqft_above','sqft_basement','yr_built',
                         'yr_renovated','lat','long','sqft_living15','sqft_lot15'] 
advanced_features_model = tc.linear_regression.create(train_data,target='price',features=advanced_features,validation_set=None)


Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 18

Number of unpacked features : 18

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.073437     | 3380697.412403     | 144449.623144                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

{'max_error': 2597844.564012507, 'rmse': 148032.5373869167}

In [20]:
# Avaliar os modelos
my_features_evaluation = my_features_model.evaluate(test_data)
advanced_features_evaluation = advanced_features_model.evaluate(test_data)
# Extrair o RMSE dos resultados de avaliação
my_features_rmse = my_features_evaluation['rmse']
advanced_features_rmse = advanced_features_evaluation['rmse']
# Subtrair o RMSE do segundo modelo pelo RMSE do primeiro modelo
print("My Feature RMSE:", my_features_evaluation)
print("Advanced RMSE:", advanced_features_evaluation )

My Feature RMSE: {'max_error': 3486584.509381705, 'rmse': 179542.4333126903}
Advanced RMSE: {'max_error': 2597844.564012507, 'rmse': 148032.5373869167}
